<a href="https://colab.research.google.com/github/pikey-msc/Python/blob/main/2022-2/Insumos/Acc_Div_SH_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Riesgo de Mercado (Simulación Histórica)

Existen trés métodos principales para medir el riesgo de mercado
* Simulación histórica
* Paramétrico
* Simulación de Monte Carlo



   **Pérdidas y Ganancias simulación histórica (sin alisado)**
   
Un enfoque más claro es suponer que tenemos:
1.   Una matriz $X_{(n+1)\times m}$ de $m$ factores de riesgo y $n+1$ observaciones. 
2.   Denotemos el vector de precios actual como $X_{00}:=(x_{0,1},x_{0,2},\ldots,x_{0,m})$. 
3.   Sea $r$ el número de instrumentos de un portafolio, entonces cada instrumento tiene una función de valuación $f_i\colon A_i \to R$ para todo $x\in X$, $i=1,\ldots, r$, donde $A_i\subset X_i$ con $\#(A_i)\leq \#(X_i)$.
4.   Sea $r$ el número de instrumentos de un portafolio, entonces cada instrumento tiene una función de valuación $f_i\colon A_i \to R$ para todo $x\in X$, $i=1,\ldots, r$, donde $A_i\subset X_i$ con $\#(A_i)\leq \#(X_i)$.
5.   Sea $M_{1\times r}=(m_1,\ldots,m_r)$ el vector de posiciones nominales de cada instrumento, es decir, el número de contratos que se tienen por instrumento $m_i\in R$ $(i=1,\ldots,r)$.

In [ ]:
!pip install yfinance
import yfinance as yf
import pandas as pd


     |████████████████████████████████| 6.4 MB 7.7 MB/s 
     |████████████████████████████████| 63 kB 1.2 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
start = '2020-01-01'

symbol = ['AAPL','MSFT','SQ','AMZN']
stock_data = yf.download(symbol,start)
# stock_data = stock_data.history(start=start)#, end=end) 
# symbol1='APPL'
# stock_data = yf.Ticker(symbol)
# stock_data = stock_data.history(start=start)#, end=end) 


[*********************100%***********************]  4 of 4 completed



##Paso 1.
 (Generación de rendimientos) Construir ${\Delta X}_{n\times m}$ que es la matriz de diferencias basados en el operador $T_j$, es decir
		$${\Delta X}_t=\Big[ T_j(\frac{x_{t,1}}{x_{t+1,1}}), T_j(\frac{x_{t,2}}{x_{t+1,2}}),\ldots, T_j(\frac{x_{t,m}}{x_{t+1,m}})\Big]\quad t=0,1,\ldots, n-1.$$

In [ ]:
# yf.download?
print(stock_data.Close.head(5))

import numpy as np
X=stock_data.Close
X0=stock_data.Close.tail(1)
DeltaX=np.log(stock_data.Close/stock_data.Close.shift(1))

print(DeltaX.head())
print(X0)


                 AAPL         AMZN        MSFT         SQ
Date                                                     
2020-01-02  75.087502  1898.010010  160.619995  63.830002
2020-01-03  74.357498  1874.969971  158.619995  63.000000
2020-01-06  74.949997  1902.880005  159.029999  62.570000
2020-01-07  74.597504  1906.859985  157.580002  64.589996
2020-01-08  75.797501  1891.969971  160.089996  67.599998
                AAPL      AMZN      MSFT        SQ
Date                                              
2020-01-02       NaN       NaN       NaN       NaN
2020-01-03 -0.009770 -0.012213 -0.012530 -0.013089
2020-01-06  0.007937  0.014776  0.002581 -0.006849
2020-01-07 -0.004714  0.002089 -0.009160  0.031774
2020-01-08  0.015958 -0.007839  0.015803  0.045548
                  AAPL         AMZN        MSFT          SQ
Date                                                       
2022-04-04  178.440002  3366.929932  314.970001  145.190002


In [ ]:
import pandas as pd
M=np.array([1000,5000,2000,100])
print(M)

#Valoración

Y0=X0.squeeze()*M

Y0



[1000 5000 2000  100]


AAPL    1.784400e+05
AMZN    1.683465e+07
MSFT    6.299400e+05
SQ      1.451900e+04
Name: 2022-04-04 00:00:00, dtype: float64

##Paso 2. 
Construir ${X^\ast}_{n\times m}$ que es la matriz de factores de riesgo simulada basada en el vector de precios actual $X_{00}$
		$$X_{t}^\ast=\Big[x_{0,1}T^{-1}_j(\Delta x_{t,1}), x_{0,2}T^{-1}_j(\Delta x_{t,2}),\ldots, x_{0,m}T^{-1}_j(\Delta x_{t,m})\Big]\quad (t=1,\ldots,n\quad j=1,2),$$
		donde $T_1(x)=\ln(x)$, $T_2(x)=x-1$.



In [ ]:
X_s=np.exp(DeltaX.squeeze())*X0.squeeze()
X_s.head

<bound method NDFrame.head of                   AAPL         AMZN        MSFT          SQ
Date                                                       
2020-01-02         NaN          NaN         NaN         NaN
2020-01-03  176.705202  3326.058600  311.048074  143.302051
2020-01-06  179.861856  3417.048670  315.784141  144.199022
2020-01-07  177.600791  3373.972055  312.098181  149.877286
2020-01-08  181.310440  3340.638732  319.986964  151.956100
...                ...          ...         ...         ...
2022-03-29  181.854345  3373.395189  319.744723  154.356358
2022-03-30  177.253458  3306.994713  313.422145  138.525747
2022-03-31  175.268089  3300.047204  309.400385  140.526512
2022-04-01  178.133419  3378.549117  316.103995  143.069960
2022-04-04  182.667861  3465.461401  320.619538  157.761844

[569 rows x 4 columns]>

## Paso 3. 
Construcción de la matriz de reevaluación del portafolio basada en los escenarios históricos $Y_{n\times r}$ de todos los instrumentos financieros, es decir
		$$Y_t=[m_1f_1(X_{t}^\ast), m_2f_2(X_{t}^\ast),\ldots, m_rf_r(X_{t}^\ast)]\quad (t=1,\ldots,n).$$
    


In [ ]:
Y_s=X_s*M.squeeze()
Y_s.head


<bound method NDFrame.head of                      AAPL          AMZN           MSFT            SQ
Date                                                                
2020-01-02            NaN           NaN            NaN           NaN
2020-01-03  176705.202400  1.663029e+07  622096.147111  14330.205063
2020-01-06  179861.856138  1.708524e+07  631568.282077  14419.902236
2020-01-07  177600.790895  1.686986e+07  624196.362322  14987.728579
2020-01-08  181310.439760  1.670319e+07  639973.927606  15195.610002
...                   ...           ...            ...           ...
2022-03-29  181854.344675  1.686698e+07  639489.445351  15435.635810
2022-03-30  177253.457791  1.653497e+07  626844.290425  13852.574731
2022-03-31  175268.089027  1.650024e+07  618800.770022  14052.651220
2022-04-01  178133.418940  1.689275e+07  632207.990521  14306.995976
2022-04-04  182667.861381  1.732731e+07  641239.075456  15776.184388

[569 rows x 4 columns]>

## Paso 4. 
Construcción de la matriz de pérdidas y ganancias del portafolio basada en los escenarios históricos $\Delta Y_{n\times r}$ de todos los instrumentos financieros, es decir
		$$\Delta Y_t=Y_t-Y_{0}=[m_1(f_1(X_{00})-f_1(X_{t}^\ast)), m_2( f_2(X_{00})-f_2(X_{t}^\ast)),\ldots, m_r(f_r(X_{00})-f_r(X_{t}^\ast))],$$
		para $(t=1,\ldots,n)$. Se puede obtener el vector de pérdidas totales ${\Delta YT}_{n\times m}$ muy fácilmente, esto es $\Delta YT=\sum_{k=1}^r Y_{kt}$ $(t=1,\ldots,n)$, incluso se puede hacer lo mismo por tipo de riesgo.


In [ ]:
PL=Y_s-Y0

cols=PL.shape[1]

PL['Total']=PL.sum(axis=1)
PL.head

<bound method NDFrame.head of                    AAPL           AMZN          MSFT           SQ  \
Date                                                                
2020-01-02          NaN            NaN           NaN          NaN   
2020-01-03 -1734.800042 -204356.659729  -7843.855330  -188.795182   
2020-01-06  1421.853696  250593.692008   1628.279636   -99.098009   
2020-01-07  -839.211546   35210.615838  -5743.640119   468.728335   
2020-01-08  2870.437318 -131455.996737  10033.925164   676.609757   
...                 ...            ...           ...          ...   
2022-03-29  3414.342234   32326.287568   9549.442910   916.635566   
2022-03-30 -1186.544650 -299676.094843  -3095.712016  -666.425513   
2022-03-31 -3171.913414 -334413.637679 -11139.232419  -466.349024   
2022-04-01  -306.583501   58095.925242   2267.988080  -212.004268   
2022-04-04  4227.858939  492657.345021  11299.073014  1257.184144   

                    Total  
Date                       
2020-01-02      

##Paso 5. 
Obtenerla medida de riesgo basado en un nivel de confianza de la matriz ${\Delta Y}_{\cdot k}$ $(k=1,\ldots,r)$ y $\Delta YT$.

Las pérdidas y ganancias vía simulación histórica sin alisado es donde se ponderan los escenarios históricos con un mismo peso para todos, es decir cada escenario tiene una probabilidad de ocurrencia de $1/n$.

In [ ]:
alpha = 0.98 #@param {type:"slider", min:0.8, max:1, step:0.001}

###Valor en Riesgo (VaR)

Es un cuantil a un $\alpha$ de confianza de la distribución de pérdidas y ganancias, generalmente se expresa en términos monetarios.

#VaR

El valor en riesgo (VaR) es una estadística utilizada para tratar de cuantificar el nivel de riesgo financiero dentro de una empresa o cartera durante un período de tiempo específico. VaR proporciona una estimación de la pérdida máxima de una posición o cartera determinada durante un período de tiempo, y puede calcularla a través de varios niveles de confianza.

Estimar el riesgo de una cartera es importante para el crecimiento del capital a largo plazo y la gestión del riesgo, en particular dentro de empresas o instituciones más grandes. El VaR generalmente se enmarca como algo como esto:

“Tenemos un VaR de cartera de 250.000 USD para el próximo mes al 95% de confianza”
Esto significa que, con un 95% de confianza, podemos decir que la pérdida de la cartera no superará los 250.000 USD en un mes.
En esta publicación, lo guiaré a través de los pasos para calcular esta métrica en una cartera de acciones.

In [ ]:
VaR=PL.quantile(1-alpha, numeric_only=False)

print(VaR)


VaRsi=[]

for i in range(0,PL.shape[1]):
  VC=PL.iloc[:,i].sort_values(ascending=True).quantile(1-alpha, interpolation = 'higher')
  VaRsi.append(VC)

print(VaRsi)

print(sum(Y0))

AAPL      -8215.669704
AMZN    -765784.448698
MSFT     -26990.346858
SQ        -1144.982959
Total   -816536.572913
Name: 0.020000000000000018, dtype: float64
[-8121.9536565445305, -739788.1953073014, -26660.67456800316, -1090.3814614255625, -786595.3656852178]
17657548.663330078


###Valor en Riesgo Condicional o "Expected Shortfall"

Es el promedio de las pérdidas que son más severas que el Valor en Riesgo ($VaR_\alpha$) a un $\alpha$ de confianza, es decir:
$$E(x|x\le VaR_\alpha)$$


In [ ]:
def VaR_Cond(x , VaR ):
  return np.mean(x[x <= VaR])

In [ ]:

VaR_C=[]

for i in range(0,PL.shape[1]):
  VC=VaR_Cond(PL.iloc[:,i],VaR[i])
  VaR_C.append(VC)


In [ ]:

print(VaR)
print(VaR_C)

pd.concat([VaR,pd.DataFrame(VaR_C).squeeze()], axis=1, ignore_index=True)

AAPL      -8215.669704
AMZN    -765784.448698
MSFT     -26990.346858
SQ        -1144.982959
Total   -816536.572913
Name: 0.020000000000000018, dtype: float64
[-12389.110031190634, -1025152.1500732518, -40925.04803283093, -1686.1279165278247, -1066975.8790856802]


,0,1
AAPL,-8215.669704,NaN
AMZN,-765784.448698,NaN
MSFT,-26990.346858,NaN
SQ,-1144.982959,NaN
Total,-816536.572913,NaN
0,NaN,-1.238911e+04
1,NaN,-1.025152e+06
2,NaN,-4.092505e+04
3,NaN,-1.686128e+03
4,NaN,-1.066976e+06


In [ ]:
def ExponentialWeighted(primer, num_esc):
  return [primer*((1-primer)**i) for i in range(num_esc)]

Sirve para generar el VaR con alisado

In [ ]:
PL.dropna(inplace = True)
PL.head()

,AAPL,AMZN,MSFT,SQ,Total
Date,,,,,
2020-01-03,-1734.800042,-204356.659729,-7843.855330,-188.795182,-214124.110283
2020-01-06,1421.853696,250593.692008,1628.279636,-99.098009,253544.727332
2020-01-07,-839.211546,35210.615838,-5743.640119,468.728335,29096.492507
2020-01-08,2870.437318,-131455.996737,10033.925164,676.609757,-117875.024497
2020-01-09,3790.210898,80794.059351,7869.823435,70.877171,92524.970855


In [ ]:
PL = PL.iloc[::-1]
PL["Esc_SA"] = ExponentialWeighted(0.05,PL.shape[0])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
PL.head()

,AAPL,AMZN,MSFT,SQ,Total,Esc_SA
Date,,,,,,
2022-04-04,4227.858939,492657.345021,11299.073014,1257.184144,509441.461119,0.050000
2022-04-01,-306.583501,58095.925242,2267.988080,-212.004268,59845.325553,0.047500
2022-03-31,-3171.913414,-334413.637679,-11139.232419,-466.349024,-349191.132536,0.045125
2022-03-30,-1186.544650,-299676.094843,-3095.712016,-666.425513,-304624.777022,0.042869
2022-03-29,3414.342234,32326.287568,9549.442910,916.635566,46206.708278,0.040725


In [ ]:
# Comprobar que sume 1
PL.Esc_SA.sum()

0.9999999999997767

In [ ]:
PL.head()

,AAPL,AMZN,MSFT,SQ,Total,Esc_SA
Date,,,,,,
2022-04-04,4227.858939,492657.345021,11299.073014,1257.184144,509441.461119,0.050000
2022-04-01,-306.583501,58095.925242,2267.988080,-212.004268,59845.325553,0.047500
2022-03-31,-3171.913414,-334413.637679,-11139.232419,-466.349024,-349191.132536,0.045125
2022-03-30,-1186.544650,-299676.094843,-3095.712016,-666.425513,-304624.777022,0.042869
2022-03-29,3414.342234,32326.287568,9549.442910,916.635566,46206.708278,0.040725


In [ ]:
def VaR_CA(x,P,alpha):
  aux = pd.DataFrame({"X":x, "Proba":P})
  aux = aux.sort_values(by = ["X"])
  aux["Acumulado"] = aux["Proba"].cumsum()
  aux2 = aux[aux["Acumulado"]>(1-alpha)]
  return aux2["X"].values[0]

In [ ]:
vector_Var = [VaR_CA(PL.iloc[:,i],PL.Esc_SA,0.98) for i in range(5)]
print(vector_Var)

[-4851.107202738407, -946451.3178738821, -23797.115299724042, -1228.0897161760458, -975708.685333298]


In [ ]:
def CVaR_CA(x,w,p=0.5):
  aux = pd.DataFrame({"X":x, "Proba":w})
  aux = aux.sort_values(by = ["X"])
  aux["Acumulado"] = aux["Proba"].cumsum()
  plist = aux["Acumulado"]/aux["Acumulado"].values[len(aux)-1]
  aux2 = aux[plist<(1-p)]
  loss = aux2["X"].values
  esc = aux2["Proba"].values
  return sum(loss*esc)/sum(esc)

In [ ]:
vector_C_Var = [CVaR_CA(PL.iloc[:,i],PL.Esc_SA,0.98) for i in range(5)]
print(vector_C_Var)

[-6549.263564845153, -1194093.0129078357, -25029.39932478193, -1570.9101864895151, -1217733.5832499403]
